# Proteins

Standard name - systematic name

STE24 - YJR117W

RCE1  - YMR274C

RAM1  - YDL090C

Do not form complexes with each other


Found three useful ones:

(first trial)
- YDR388W (Cytoskeleton one):  community 15, betweenness -> not in human

(second trial)
- YDR388W (Cytoskeleton one): community 12; betweenness, information, PCA; general partition -> not in human

- YLR025W (Endosomal one): community 8; information, subgraph, PCA; general partition

- YMR037C (DNA binding transcription): community 0, betweenness, information; SA partition

- YNL298W: communitty 11, information; SA partition

# Data Preparation

## Packages

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # for interactive plot
import pandas as pd

# for Louvain algo
from networkx.algorithms import community
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import k_clique_communities

from community import community_louvain

# for copying graph
import copy

# basic settings for plotting figures
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)

## Functions

In [2]:
'''
STE24 - YJR117W
RCE1  - YMR274C
RAM1  - YDL090C
'''
# Proteins that we focus on
def print_protein(str):
    if str == 'YJR117W':
        return 'STE24(YJR117W)'
    elif str == 'YMR274C':
        return 'RCE1(YMR274C)'
    elif str == 'YDL090C':
        return 'RAM1(YDL090C)'

In [3]:
def centrality_calculater(G):
    deg = nx.degree_centrality(G)
    close = nx.closeness_centrality(G, u=None, distance=None, wf_improved=True)
    eig = nx.eigenvector_centrality(G, max_iter=150, tol=1e-06, nstart=None, weight=None)
    bet = nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=2013)
    sub = nx.subgraph_centrality(G)
    info = nx.information_centrality(G, weight=None, solver='lu')

    names = list(deg.keys())
    degree_list = np.array(list(deg.values()))
    closeness_list = np.array(list(close.values()))
    eigenvector_list = np.array(list(eig.values()))
    betweenness_list = np.array(list(bet.values()))
    subgraph_list = np.array(list(sub.values()))
    information_list = np.array(list(info.values()))

    centrality_df = pd.DataFrame()
    centrality_df['Names'] = names
    centrality_df['Degree'] = degree_list
    centrality_df['Closeness'] = closeness_list
    centrality_df['Eigenvector'] = eigenvector_list
    centrality_df['Betweenness'] = betweenness_list
    centrality_df['Subgraph'] = subgraph_list
    centrality_df['Information'] = information_list


    colnames = list(centrality_df.columns)
    transformed = pd.DataFrame()
    for col in colnames:
        arr = np.array(centrality_df[col])
        if col == "Betweenness":
            arr += 1e-10
            arr = np.log(arr)
        elif col != "Closeness" and col != "Information" and col!= "Names":
            arr += 1e-20
            arr = np.log(arr)

        transformed[col] = arr


    importance = []
    for i in range(len(list(transformed["Names"]))):
        importance_score = 0.21*transformed["Degree"][i] + 0.21*transformed["Closeness"][i] + 0.2*transformed["Eigenvector"][i] + 0.16*transformed["Betweenness"][i] + 0.22*transformed["Information"][i]

        importance.append(importance_score)

    transformed["Importance"] = importance

    importance_dict = {}
    for i in range(len(importance)):
        importance_dict[transformed["Names"][i]] = transformed["Importance"][i]

    # sorted_importance  = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

    return importance_dict

In [4]:
# find the second last value and its corresponding key in a dictionary
def second_max(dict):
    all_value = list( dict.values() )
    all_value.sort(reverse=True)

    second_largest_value = all_value[1]

    for key, value in dict.items():
        if value == second_largest_value:
            return key, value  

## Given proteins

In [5]:
# Store proteins into a list
all_proteins = ['YJR117W', 'YMR274C', 'YDL090C']

## Store and modify network data

In [6]:
# Store the network as G
# Storing format - dictionary: { u: {v1: {'weight': 123.0}, v2: {'weight': 456.0} } }; u, v - str
G = nx.read_weighted_edgelist("cleanData.txt",comments="#",nodetype=str)

In [7]:
# Remove edges with confidence score <= 700
threshold = 700
for edge in G.edges:     
    # edge (u, v), an edge between node u and node v
    u = edge[0]
    v = edge[1]
    weight = G[u][v]['weight']

    if weight <= threshold:
        G.remove_edge(u,v)

In [8]:
# only consider the largest component to get rid of the nodes with 0 degree
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc)

# Community analysis

## Louvain

In [9]:
'''
Louvain group modularity algorithm

- Intro:
 https://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008
 https://en.wikipedia.org/wiki/Louvain_modularity
 Based on modularity optimization

- Documentation: 
 https://python-louvain.readthedocs.io/en/latest/api.html

- Why:
 Good enough for the starting stage
 O(n log n) -> very efficient to build, especially when the dataset is large
 Modularity matches Biological meaning: internal dense connection - more interaction among proteins - have similar function
'''

# {protein1: community, protein2: community, ...} - {string : int}
# weight = 'None' as unweighted graph
# random_state: int; seed
# resolution: int; control the community size

# partition_Louvain = community_louvain.best_partition(G, weight = 'None', random_state = 2013, resolution = 1.0) 

# try 100 times to find the partition with highest modularity
loop_time = 100
all_modularity = []
max_modularity = float('-inf')
chosen_seed = -1


for i in range(loop_time):
    seed = i
    temp_partition = community_louvain.best_partition(G, weight = 'None', random_state = seed, resolution = 1.0)
    # https://python-louvain.readthedocs.io/en/latest/api.html
    temp_modularity = community_louvain.modularity(temp_partition, G, weight='None')
    
    all_modularity.append(temp_modularity) # store only for future reference
    
    if temp_modularity >= max_modularity:
        max_modularity = temp_modularity
        partition_Louvain = temp_partition
        chosen_seed = i

In [10]:
print(f'The maximum modularity is: {max_modularity}; while the minimum modularity is {min(all_modularity)}')
print(f'The seed that achieve the max modularity is: {chosen_seed}')

total_communities = set()
for community in partition_Louvain.values():
    total_communities.add(community)
print(f'Total number of communities: {len(total_communities)}')

The maximum modularity is: 0.6501388566754679; while the minimum modularity is 0.6395302586804178
The seed that achieve the max modularity is: 53
Total number of communities: 20


## Communities of the chosen proteins

In [11]:
home_community = set()

for protein in all_proteins:
    home_community.add(partition_Louvain[protein])
    print(f'{print_protein(protein)} is in community {partition_Louvain[protein]}')

STE24(YJR117W) is in community 11
RCE1(YMR274C) is in community 11
RAM1(YDL090C) is in community 11


## Adjacent communities

### Directly connected to the three poteins

In [12]:
# get the list of proteins in a same community as the target ones (included target ones)
proteins_in_target_community = ['YJR117W', 'YMR274C', 'YDL090C']
for protein, community in partition_Louvain.items():
    if community == partition_Louvain['YJR117W']:
        proteins_in_target_community.append(protein)

adjacent_communities = set() # {int}

for protein in proteins_in_target_community:
    neighbors = G.neighbors(protein)
    
    for neighbor in neighbors:
        community = partition_Louvain[neighbor]

        if community != partition_Louvain[protein]:
            adjacent_communities.add(community)

print(f'Number of adjacent communities: {len(adjacent_communities)}')
print(f'Adjacent communities are {adjacent_communities}')

Number of adjacent communities: 18
Adjacent communities are {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19}


In [13]:
# union all communities: home community + adjacent community
all_communities = home_community.union(adjacent_communities)
print(f'All communities to look at are {all_communities}')

All communities to look at are {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}


## Store communities as its own group

In [14]:
'''
store communities in a dict: 
{4: subgraph of corresponding nodes, 6: subgraph of corresponding nodes, ...}
'''

# Store communities and their corresponding protein name as a dictionary
# {0: [proteins], 8: [proteins]}

community_proteinList = {}

for community_num in all_communities:
    community_proteinList[community_num] = []

for protein, community_num in partition_Louvain.items():
    if community_num in all_communities:
        community_proteinList[community_num].append(protein)

# Dictionary storing community and corresponding nodes' subgraph information
clusters = {}
for community_num, protein_list in community_proteinList.items():
    clusters[community_num] = G.subgraph(protein_list)

## Communities size and number of edges

In [15]:
for community_num, protein_list in community_proteinList.items():
    print(f'The size of Community {community_num} is: {len(protein_list)}')
    print(f'The number of edges of Community {community_num} is: {clusters[community_num].number_of_edges()}\n')

The size of Community 0 is: 212
The number of edges of Community 0 is: 1961

The size of Community 1 is: 655
The number of edges of Community 1 is: 5127

The size of Community 2 is: 230
The number of edges of Community 2 is: 1014

The size of Community 3 is: 116
The number of edges of Community 3 is: 500

The size of Community 4 is: 22
The number of edges of Community 4 is: 35

The size of Community 5 is: 283
The number of edges of Community 5 is: 2116

The size of Community 6 is: 227
The number of edges of Community 6 is: 1070

The size of Community 7 is: 332
The number of edges of Community 7 is: 2371

The size of Community 8 is: 60
The number of edges of Community 8 is: 329

The size of Community 9 is: 158
The number of edges of Community 9 is: 562

The size of Community 10 is: 280
The number of edges of Community 10 is: 786

The size of Community 11 is: 541
The number of edges of Community 11 is: 2447

The size of Community 12 is: 288
The number of edges of Community 12 is: 1321

T

## Select reasonable adjacent communities

### Rank normalised number of connections from target community to adjacent communities

In [16]:
# direct connction: normalised number of connections between the target community and adjacent communities
# direction score: number of connections between the target community and adjacent communities / total number of connections of the adjacent community 
direct_connection = {}
for community in adjacent_communities:
    direct_connection[community] = 0

for protein in proteins_in_target_community:
    for community, nodeList in community_proteinList.items():
        if community in adjacent_communities:
            for node in nodeList:
                if node in G.neighbors(protein):
                    direct_connection[community] += 1

# normalised the number of connection
for community, n_connection in direct_connection.items():
    print(f'Community {community} has number of connection: {direct_connection[community]}')
   
    direct_connection[community] = n_connection / len(community_proteinList[community])   
    print(f'Community {community} has normalised connection score: {direct_connection[community]}')
    print('')

Community 0 has number of connection: 92
Community 0 has normalised connection score: 0.4339622641509434

Community 1 has number of connection: 250
Community 1 has normalised connection score: 0.3816793893129771

Community 2 has number of connection: 54
Community 2 has normalised connection score: 0.23478260869565218

Community 3 has number of connection: 23
Community 3 has normalised connection score: 0.19827586206896552

Community 4 has number of connection: 1
Community 4 has normalised connection score: 0.045454545454545456

Community 5 has number of connection: 171
Community 5 has normalised connection score: 0.6042402826855123

Community 6 has number of connection: 83
Community 6 has normalised connection score: 0.3656387665198238

Community 7 has number of connection: 34
Community 7 has normalised connection score: 0.10240963855421686

Community 8 has number of connection: 25
Community 8 has normalised connection score: 0.4166666666666667

Community 9 has number of connection: 15

In [17]:
# sort the number of direct connection with value

#sorted_direct_connection = {k: v for k, v in sorted(direct_connection.items(), key=lambda item: item[1])}
sorted_direct_connection = dict(sorted(direct_connection.items(), key=lambda item: item[1], reverse = True))

for key, items in sorted_direct_connection.items():
    print(f'Community {key} has score: {items} (after sorted)')


# choose top adjacent communities
n_adjacent = 4
chosen_adjacent = list( sorted_direct_connection.keys() )[:n_adjacent]

print(f'\n The chosen adjacent communities (top {n_adjacent}) are: {chosen_adjacent}')

Community 5 has score: 0.6042402826855123 (after sorted)
Community 12 has score: 0.4548611111111111 (after sorted)
Community 0 has score: 0.4339622641509434 (after sorted)
Community 8 has score: 0.4166666666666667 (after sorted)
Community 1 has score: 0.3816793893129771 (after sorted)
Community 6 has score: 0.3656387665198238 (after sorted)
Community 16 has score: 0.3413173652694611 (after sorted)
Community 14 has score: 0.26011560693641617 (after sorted)
Community 19 has score: 0.25 (after sorted)
Community 13 has score: 0.2413793103448276 (after sorted)
Community 2 has score: 0.23478260869565218 (after sorted)
Community 3 has score: 0.19827586206896552 (after sorted)
Community 15 has score: 0.10666666666666667 (after sorted)
Community 7 has score: 0.10240963855421686 (after sorted)
Community 9 has score: 0.0949367088607595 (after sorted)
Community 17 has score: 0.049689440993788817 (after sorted)
Community 4 has score: 0.045454545454545456 (after sorted)
Community 10 has score: 0.014

## Representative nodes in the found communities

### Representative nodes: (closeness), betweeness centrality, (eigenvalue), information, subgraph

In [18]:
# {community: protein, community: protein, xxx}
closeness_representatives = {}
betweenness_representatives = {}
eigenvalue_representatives = {}
information_representatives = {}
subgraph_representatives = {}
PCA_representatives = {}

# second representatives
closeness_second_representatives = {}
betweenness_second_representatives = {}
eigenvalue_second_representatives = {}
information_second_representatives = {}
subgraph_second_representatives = {}
PCA_second_representatives = {}

communities_to_focus = home_community.union(chosen_adjacent)

for community in communities_to_focus:
    # # closeness {node: score, node: score, xxx}
    # closeness = {}
    # closeness = nx.closeness_centrality(clusters[community], u=None, distance=None, wf_improved=True)
    # max_closeness = max(closeness.values()) # max because in networkX it is reciprocal to the average distance
    
    # for protein in closeness:
    #     if closeness[protein] == max_closeness:
    #         closeness_representatives[community] = protein

    # ## second representatives
    # found_representative, score = second_max(closeness)
    # closeness_second_representatives[community] = found_representative
    

    # betweenness
    # whether to set seed?
    betweenness = {}
    betweenness = nx.betweenness_centrality(clusters[community], k=None, normalized=True, weight=None, endpoints=False, seed=2013) 
    max_betweenness = max(betweenness.values())
    
    for protein in betweenness:
        if betweenness[protein] == max_betweenness:
            betweenness_representatives[community] = protein

    ## second representatives
    found_representative, score = second_max(betweenness)
    betweenness_second_representatives[community] = found_representative

    # # eigenvalue
    # # how to set max_iter
    # eigenvalue = {}
    # eigenvalue = nx.eigenvector_centrality(clusters[community], max_iter=150, tol=1e-06, nstart=None, weight=None)
    # max_eigenvalue = max(eigenvalue.values())
    
    # for protein in eigenvalue:
    #     if eigenvalue[protein] == max_eigenvalue:
    #         eigenvalue_representatives[community] = protein 

    # ## second representatives
    # found_representative, score = second_max(eigenvalue)
    # eigenvalue_second_representatives[community] = found_representative

    # information
    information = {}
    information = nx.information_centrality(clusters[community], weight=None, solver='lu')
    max_information = max(information.values())
    
    for protein in information:
        if information[protein] == max_information:
            information_representatives[community] = protein 

    ## second representatives
    found_representative, score = second_max(information)
    information_second_representatives[community] = found_representative

    
    # subgraph
    subgraph = {}
    subgraph = nx.subgraph_centrality(clusters[community])
    max_subgraph = max(subgraph.values())
    
    for protein in subgraph:
        if subgraph[protein] == max_subgraph:
            subgraph_representatives[community] = protein 

    ## second representatives
    found_representative, score = second_max(subgraph)
    subgraph_second_representatives[community] = found_representative

    
    # PCA: 
    PCA = {}
    PCA = centrality_calculater(clusters[community])
    
    max_pca = max(PCA.values())

    for protein in PCA:
        if PCA[protein] == max_pca:
            PCA_representatives[community] = protein

    ## second representatives
    found_representative, score = second_max(PCA)
    PCA_second_representatives[community] = found_representative

In [19]:
# print results and put them in a set
representatives = set()
second_representatives = set()

# print('Closeness representatives:')
# for communityNum, protein in closeness_representatives.items():
#     representatives.add(protein)
#     print(f'  - Community {communityNum}: {protein}')
# print('')

# print('Closeness second representatives:')
# for communityNum, protein in closeness_second_representatives.items():
#     second_representatives.add(protein)
#     print(f'  - Community {communityNum}: {protein}')
# print('')


print('Betweenness representatives:')
for communityNum, protein in betweenness_representatives.items():
    representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')

print('Betweenness second representatives:')
for communityNum, protein in betweenness_second_representatives.items():
    second_representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')


# print('Eigenvalue representatives:')
# for communityNum, protein in eigenvalue_representatives.items():
#     representatives.add(protein)
#     print(f'  - Community {communityNum}: {protein}')
# print('')

# print('Eigenvalue second representatives:')
# for communityNum, protein in eigenvalue_second_representatives.items():
#     second_representatives.add(protein)
#     print(f'  - Community {communityNum}: {protein}')
# print('')


print('Information representatives:')
for communityNum, protein in information_representatives.items():
    representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')

print('Inforrmation second representatives:')
for communityNum, protein in information_second_representatives.items():
    second_representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')


print('Subgraph representatives:')
for communityNum, protein in subgraph_representatives.items():
    representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')

print('Subgraph second representatives:')
for communityNum, protein in subgraph_second_representatives.items():
    second_representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')


print('PCA representatives:')
for communityNum, protein in PCA_representatives.items():
    representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')

print('PCA second representatives:')
for communityNum, protein in PCA_second_representatives.items():
    second_representatives.add(protein)
    print(f'  - Community {communityNum}: {protein}')
print('')

print(f'All representatives: {representatives}\n')
print(f'All second representatives: {second_representatives}\n')

Betweenness representatives:
  - Community 0: YER095W
  - Community 5: YBR010W
  - Community 8: YGL045W
  - Community 11: YDR477W
  - Community 12: YDR388W

Betweenness second representatives:
  - Community 0: YML032C
  - Community 5: YOL012C
  - Community 8: YHL027W
  - Community 11: YLR113W
  - Community 12: YDL077C

Information representatives:
  - Community 0: YER095W
  - Community 5: YBR010W
  - Community 8: YLR025W
  - Community 11: YLR113W
  - Community 12: YDR388W

Inforrmation second representatives:
  - Community 0: YML032C
  - Community 5: YNL030W
  - Community 8: YCL008C
  - Community 11: YHL007C
  - Community 12: YLR262C

Subgraph representatives:
  - Community 0: YER095W
  - Community 5: YBR010W
  - Community 8: YLR025W
  - Community 11: YLR113W
  - Community 12: YBL037W

Subgraph second representatives:
  - Community 0: YML032C
  - Community 5: YOL012C
  - Community 8: YCL008C
  - Community 11: YBL016W
  - Community 12: YOL062C

PCA representatives:
  - Community 0: YER0

### Shotest paths between target proteins and representatives

In [20]:
for target_protein in all_proteins:
    print('- ' + print_protein(target_protein) + ':')
    for rep in representatives:
        path = nx.shortest_path(G, source=target_protein, target=rep, weight=None, method='dijkstra')
        print(f"Shortest path to {rep}: {path}")
    print('')

- STE24(YJR117W):
Shortest path to YGL045W: ['YJR117W', 'YKL209C', 'YFL026W', 'YFR022W', 'YGL045W']
Shortest path to YDR477W: ['YJR117W', 'YDL090C', 'YNL098C', 'YDR477W']
Shortest path to YER095W: ['YJR117W', 'YLR292C', 'YMR186W', 'YER095W']
Shortest path to YDR388W: ['YJR117W', 'YKL209C', 'YMR297W', 'YJL154C', 'YDR388W']
Shortest path to YBL037W: ['YJR117W', 'YMR214W', 'YDR320C', 'YGR167W', 'YBL037W']
Shortest path to YLR113W: ['YJR117W', 'YDL090C', 'YNL098C', 'YLR113W']
Shortest path to YBR010W: ['YJR117W', 'YLR292C', 'YMR186W', 'YBR010W']
Shortest path to YLR025W: ['YJR117W', 'YKL209C', 'YMR297W', 'YNR006W', 'YLR025W']

- RCE1(YMR274C):
Shortest path to YGL045W: ['YMR274C', 'YNL098C', 'YFR022W', 'YGL045W']
Shortest path to YDR477W: ['YMR274C', 'YNL098C', 'YDR477W']
Shortest path to YER095W: ['YMR274C', 'YDL135C', 'YOR127W', 'YNL298W', 'YER095W']
Shortest path to YDR388W: ['YMR274C', 'YDL135C', 'YOR127W', 'YDL117W', 'YDR388W']
Shortest path to YBL037W: ['YMR274C', 'YNL041C', 'YJL004C

# random check

In [21]:
print('YLR025W' in list(G.nodes))

True
